# Overview
The purpose of this script is to quickly analyze the trip report that I web-scraped from my dispatch terminal (TOD report) and the settlement report generated by the corporate payroll team (RTS report) and compare them to ensure that there are no discrepancies slipping through the cracks and creating a situation where a driver is over/underpaid for any period worked

In [1]:
# Importing needed packages
import numpy as np # linear algebra
import pandas as pd # data processing

print('Setup Complete')

Setup Complete


In [2]:
# Creating the main dataframes that are going to be worked with each week to get ready for cleaning the Ready to Settle Report
date = input('Enter RTS report date in M-D-YYYY format')
RTSFileName = '/kaggle/input/rtsreport/ReadyToSettleReport_{}.xlsx'.format(date)

RTS = pd.read_excel(RTSFileName)
TOD = pd.read_excel('/kaggle/input/todreport/Universal Settlement Processing.xlsx')

TOD['orderCode'] = TOD['orderCode'].astype('int')
RTS['orderCode'] = RTS['orderCode'].astype('int')

Enter RTS report date in M-D-YYYY format 6-30-2023


From this point, the RTS needs transformed to properly compare with the TOD report that I am able to scrape from the dispatch board

In [3]:
RTS = RTS.drop(['codeDescription','settlementStatus'], axis=1)
RTS['payRate'] = pd.to_numeric(RTS['payRate'])
TOD['payRate'] = pd.to_numeric(TOD['payRate'])

In [4]:
# Getting the RTS data into the same by-order/driver format that the TOD data is in for comparison
RTSpay = []
RTSpay = RTS.groupby(['orderCode','vendorName'])['payRate'].sum()
RTS = RTSpay.to_frame().reset_index()
RTS = RTS.set_index('orderCode')
print(RTS)

TODpay = []
TODpay = TOD.groupby(['orderCode','vendorName'])['payRate'].sum()
TOD = TODpay.to_frame().reset_index()
TOD = TOD.set_index('orderCode')

print(TOD)

                            vendorName  payRate
orderCode                                      
220017444                   Kyle Tyler    35.00
230011918            David Kaufhold II   274.26
230012563                Daniel Kebebu   373.10
230012566                  Sean Wilson   613.90
230012569             R&T TRUCKING LLC   613.90
...                                ...      ...
230016334            Dennis Helton Jr.   105.00
230016395             Bhupinder Sandhu   371.00
230016409            Dennis Helton Jr.   105.00
230016447  Bernard & Sons trucking LLC   105.00
230016480            SMF Transport LLC  1800.00

[124 rows x 2 columns]
                            vendorName  payRate
orderCode                                      
220017444                   Kyle Tyler    35.00
230011918            David Kaufhold II   274.26
230012563                Daniel Kebebu   373.10
230012566                  Sean Wilson   613.90
230012569             R&T TRUCKING LLC   613.90
...             

In [5]:
# Grabbing POs with discrepencies between the TOD and RTS reports
Disc = []
Disc = pd.concat([RTS,TOD]).drop_duplicates(keep=False)
ProbPO = []
ProbPO = Disc.index.tolist()
ProbPO

[230013351,
 230014851,
 230014901,
 230014909,
 230015218,
 230015418,
 230015794,
 230015869,
 230015938,
 230015990,
 230016164,
 230016255,
 230016282,
 230016306,
 230016480,
 230014872,
 230014873,
 230014901,
 230015035,
 230015218,
 230015418,
 230015794,
 230015869,
 230015990,
 230016164,
 230016282,
 230016306,
 230016406]

In [6]:
# Grabbing issue POs in each report
inTOD = []
inTOD = TOD.loc[TOD.index.intersection(ProbPO)].reset_index()

inRTS = []
inRTS = RTS.loc[RTS.index.intersection(ProbPO)].reset_index()

# Concatenating the discrepencies into 1 dataframe for export
vert_concat = []
vert_concat = pd.concat([inRTS,inTOD], axis=1)

ProbPOs = vert_concat.to_csv('ProbPOs.csv', header = False)

vert_concat

,orderCode,vendorName,payRate,orderCode,vendorName,payRate
0,230013351,Jedidiah Zurmehly,260.40,230014872.0,David Kaufhold II,292.30
1,230014851,Jedidiah Zurmehly,256.48,230014873.0,David Kaufhold II,282.50
2,230014873,David Kaufhold II,292.60,230014901.0,Sean Wilson,648.90
3,230014901,Sean Wilson,648.95,230015035.0,David Rom,213.90
4,230014909,Jedidiah Zurmehly,105.00,230015035.0,bdarling trucking,87.50
5,230015035,David Rom,213.90,230015218.0,BIG DAWG TRUCKING LLC,328.70
6,230015218,BIG DAWG TRUCKING LLC,328.70,230015418.0,Daniel Kebebu,422.10
7,230015418,Daniel Kebebu,211.05,230015794.0,bdarling trucking,668.19
8,230015794,bdarling trucking,668.19,230015869.0,Bhupinder Sandhu,490.70
9,230015869,Bhupinder Sandhu,490.70,230015938.0,ALNABALY GROUPS LLC,552.30


In [7]:
# Getting a current expected Gross Pay for each Vendor
GrossPay_df = []
GrossPay_df = RTS.loc[:,['vendorName','payRate']]
GrossPay = []
GrossPay = GrossPay_df.groupby('vendorName').sum()
GrossPay_csv_data = GrossPay.to_csv('driverGrossPay.csv',index=True)
GrossPay

,payRate
vendorName,
BIG DAWG TRUCKING LLC,2187.61
Bernard & Sons trucking LLC,3266.53
Bhupinder Sandhu,2197.80
Daniel Kebebu,1629.60
David Kaufhold II,2733.02
David Rom,1962.70
Dennis Helton Jr.,2313.42
Jedidiah Zurmehly,621.88
Kenneth Watkins,2205.00


In [8]:
print('''Run Complete.''')

Run Complete.
